In [ ]:
import numpy as np
import pandas as pd
from nearcorrmat.functions import *

In [ ]:
ERR_MAX = 1e-6
ITE_MAX = 10000

# Von Neumann alternative projection for the primal

In [ ]:
def update_neumann(version, prim_2):
    if version == 0:
        prim_1 = proj_cone(prim_2)
        updated = (prim_1, proj_unit_diag(prim_1))
    elif version == 1:
        prim_1 = proj_unit_diag(prim_2)
        updated = (prim_1, proj_cone(prim_1))
    return updated

In [ ]:
def proj_neumann(mat, version, err_max=1e-6, ite_max=10_000):
    err = err_max + 1
    ite = 0
    prim_2 = mat.copy()
    errors = list()
    while err > err_max and ite < ite_max:
        prim_1, prim_2 = update_neumann(version, prim_2)
        err = norm(prim_1 - prim_2)
        errors.append(err)
        ite += 1
    return 0.5 * (prim_1 + prim_2), errors

In [ ]:
%%time
projection, errors = proj_neumann(laplace, 0, ERR_MAX, ITE_MAX)
pd.DataFrame(errors).apply(np.log).plot()

In [ ]:
%%time
mat = gen_sym_psd(100, 10, 50)
projection, errors = proj_neumann(mat, 0, ERR_MAX, ITE_MAX)
log_errs = pd.DataFrame(errors).apply(np.log)
log_errs.plot()
axis = (log_errs.shift() / log_errs).plot()
axis.set_ylim(0, 2)

In [ ]:
%%timeit
mat = gen_sym_psd(100, None, 50)
projection, errors = proj_neumann(mat, 0, ERR_MAX, ITE_MAX)

In [ ]:
%%time
projection, errors = proj_neumann(laplace, 1, ERR_MAX, ITE_MAX)
pd.DataFrame(errors).apply(np.log).plot()

In [ ]:
%%time
mat = gen_sym_psd(100, 11, 50)
projection, errors = proj_neumann(mat, 1, ERR_MAX, ITE_MAX)
log_errs = pd.DataFrame(errors).apply(np.log)
log_errs.plot()
axis = (log_errs.shift() / log_errs).plot()
axis.set_ylim(0, 2)

In [ ]:
%%timeit
mat = gen_sym_psd(100, None, 50)
projection, errors = proj_neumann(mat, 1, ERR_MAX, ITE_MAX)

# Classical Dykstra's projection algorithm for the primal does not converge

In [ ]:
def update_dykstra(version, prim_2, dual):
    if version == 0:
        prim_1 = proj_cone(prim_2 - dual)
        updated = (prim_1, proj_unit_diag(prim_1 + dual))
    elif version == 1:
        prim_1 = proj_unit_diag(prim_2 - dual)
        updated = (prim_1, proj_cone(prim_1 + dual))
    return updated

In [ ]:
def proj_dykstra(mat, version, err_max=1e-6, ite_max=10_000):
    err = err_max + 1
    ite = 0
    prim_2 = mat.copy()
    dual = np.zeros(mat.shape)
    errors = list()
    while err > err_max and ite < ite_max:
        prim_1, prim_2_new = update_dykstra(version, prim_2, dual)
        dual = dual + prim_1 - prim_2_new
        err = norm(prim_1 - prim_2_new)
        errors.append(err)

        prim_2 = prim_2_new.copy()
        ite += 1
    return 0.5 * (prim_1 + prim_2), errors

In [ ]:
mat = gen_sym_psd(100, 12, 50)
projection, errors = proj_dykstra(mat, 0, ERR_MAX, ITE_MAX)
pd.DataFrame(errors).apply(np.log).plot()

In [ ]:
mat = gen_sym_psd(100, None, 50)
projection, errors = proj_dykstra(mat, 1, ERR_MAX, ITE_MAX)
pd.DataFrame(errors).apply(np.log).plot()

# Modified Dykstra's projection for the primal

In [ ]:
%%time
projection, errors = nearest_corr(
    laplace, 'grad', ERR_MAX, ITE_MAX)
pd.DataFrame(errors).apply(np.log).plot()

In [ ]:
%%time
mat = gen_sym_psd(100, 13, 50)
projection, errors = nearest_corr(
    mat, 'grad', ERR_MAX, ITE_MAX)
log_errs = pd.DataFrame(errors).apply(np.log)
log_errs.plot()
axis = (log_errs.shift() / log_errs).plot()
axis.set_ylim(0, 2)

In [ ]:
%%timeit
mat = gen_sym_psd(100, None, 50)
projection, errors = nearest_corr(
    mat, 'grad', ERR_MAX, ITE_MAX)

# ADMM for the primal

In [ ]:
%%time
projection, errors = nearest_corr(
    laplace, 'admm_v0', ERR_MAX, ITE_MAX)
pd.DataFrame(errors).apply(np.log).plot()

In [ ]:
%%time
mat = gen_sym_psd(100, 15, 50)
projection, errors = nearest_corr(
    mat, 'admm_v0', ERR_MAX, ITE_MAX)
log_errs = pd.DataFrame(errors).apply(np.log)
log_errs.plot()
axis = (log_errs.shift() / log_errs).plot()
axis.set_ylim(0, 2)

In [ ]:
%%timeit
mat = gen_sym_psd(100, None, 50)
projection, errors = nearest_corr(
    mat, 'admm_v0', ERR_MAX, ITE_MAX)

In [ ]:
%%time
projection, errors = nearest_corr(
    laplace, 'admm_v1', ERR_MAX, ITE_MAX)
pd.DataFrame(errors).apply(np.log).plot()

In [ ]:
%%time
mat = gen_sym_psd(100, 16, 50)
projection, errors = nearest_corr(
    mat, 'admm_v1', ERR_MAX, ITE_MAX)
log_errs = pd.DataFrame(errors).apply(np.log)
log_errs.plot()
axis = (log_errs.shift() / log_errs).plot()
axis.set_ylim(0, 2)

In [ ]:
%%timeit
mat = gen_sym_psd(100, None, 50)
projection, errors = nearest_corr(
    mat, 'admm_v1', ERR_MAX, ITE_MAX)

In [ ]:
ITE_MAX = 1000

# BFGS for the dual

In [ ]:
%%time
projection, errors = nearest_corr(laplace, 'bfgs', ERR_MAX, ITE_MAX)
pd.DataFrame(errors).apply(np.log).plot()

In [ ]:
%%time
mat = gen_sym_psd(100, 17, 50)
projection, errors = nearest_corr(mat, 'bfgs', ERR_MAX, ITE_MAX)
log_errs = pd.DataFrame(errors).apply(np.log)
log_errs.plot()
axis = (log_errs.shift() / log_errs).plot()
axis.set_ylim(0, 2)

In [ ]:
%%timeit
mat = gen_sym_psd(100, None, 50)
projection, errors = nearest_corr(mat, 'bfgs', ERR_MAX, ITE_MAX)

# L-BFGS for the dual

In [ ]:
%%time
projection, errors = nearest_corr(
    laplace, 'l_bfgs', ERR_MAX, ITE_MAX, memory=10)
pd.DataFrame(errors).apply(np.log).plot()

In [ ]:
%%time
mat = gen_sym_psd(100, 18, 50)
projection, errors = nearest_corr(
    mat, 'l_bfgs', ERR_MAX, ITE_MAX, memory=10)
log_errs = pd.DataFrame(errors).apply(np.log)
log_errs.plot()
axis = (log_errs.shift() / log_errs).plot()
axis.set_ylim(0, 2)

In [ ]:
%%timeit
mat = gen_sym_psd(100, None, 50)
projection, errors = nearest_corr(
    mat, 'l_bfgs', ERR_MAX, ITE_MAX, memory=10)

# Newton for the dual

In [ ]:
%%time
projection, errors = nearest_corr(laplace, 'newton', ERR_MAX , ITE_MAX)
pd.DataFrame(errors).apply(np.log).plot()

In [ ]:
%%time
mat = gen_sym_psd(100, 19, 50)
projection, errors = nearest_corr(mat, 'newton', ERR_MAX, ITE_MAX)
log_errs = pd.DataFrame(errors).apply(np.log)
log_errs.plot()
axis = (log_errs.shift() / log_errs).plot()
axis.set_ylim(0, 2)

In [ ]:
%%timeit
mat = gen_sym_psd(100, None, 50)
projection, errors = nearest_corr(mat, 'newton', ERR_MAX, ITE_MAX)